In [2]:
import sys
import os
import requests
import torch
import pandas as pd
import numpy as np
import joblib
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QComboBox, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QFrame, QScrollArea, QGridLayout, QPushButton, QMessageBox
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QPainter, QColor, QPen, QFont, QBrush
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

class KarnatakaPowerDashboard(QMainWindow):
    def __init__(self):
        super().__init__()
        self.API_KEY = "7340acbc6a0ed54052244c85b5ed57a9"
        self.districts = [
            "Bagalkot", "Ballari", "Belagavi", "Bengaluru Rural", 
            "Bengaluru Urban", "Bidar", "Chamarajanagar", "Chikkaballapur",
            "Chikkamagaluru", "Chitradurga", "Dakshina Kannada", "Davanagere",
            "Dharwad", "Gadag", "Hassan", "Haveri", "Kalaburagi", "Kodagu",
            "Kolar", "Koppal", "Mandya", "Mysuru", "Raichur", "Ramanagara",
            "Shivamogga", "Tumakuru", "Udupi", "Uttara Kannada", "Vijayapura",
            "Yadgir", "Vijayanagara"
        ]
        self.setup_encoders_and_scalers()
        self.setup_ui()
        self.load_model()

    def setup_encoders_and_scalers(self):
        try:
            self.scaler_x = joblib.load('c:/Users/dhany/Smart-Grid/scalerx.pkl')
            self.scaler_y = joblib.load('c:/Users/dhany/Smart-Grid/scalery.pkl')
            self.le_city = LabelEncoder()
            self.le_state = LabelEncoder()
            self.le_weather = LabelEncoder()
            self.le_city.fit(self.districts)
            self.le_state.fit(['Karnataka'])
            self.le_weather.fit(['Clear', 'Clouds', 'Rain', 'Thunderstorm', 'Mist', 'Haze', 'Fog'])
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to load scalers: {str(e)}")
            sys.exit(1)

    def setup_ui(self):
        self.setWindowTitle("Karnataka Power Supply Dashboard")
        self.setGeometry(100, 100, 1400, 900)
        self.setStyleSheet("""
            QMainWindow {background-color: #E8EAF6;}
            QLabel {color: #1A237E; font-size: 12px;}
            QComboBox {
                padding: 8px;
                border: 2px solid #3F51B5;
                border-radius: 5px;
                background: white;
                min-width: 200px;
                font-size: 14px;
            }
            QFrame {
                border-radius: 10px;
                background-color: white;
                margin: 5px;
                box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.2);
            }
            QScrollArea {
                border: none;
                background-color: transparent;
            }
            QPushButton {
                background-color: #3F51B5;
                color: white;
                padding: 8px 15px;
                border-radius: 5px;
                font-size: 14px;
            }
            QPushButton:hover {
                background-color: #303F9F;
            }
        """)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # Header
        header = QFrame()
        header.setStyleSheet("""
            QFrame {
                background-color: #3F51B5;
                padding: 15px;
                margin: 0px;
                border-radius: 0px;
            }
            QLabel {
                color: white;
                font-size: 24px;
                font-weight: bold;
            }
        """)
        header_layout = QHBoxLayout(header)
        title = QLabel("Karnataka Power Supply Dashboard")
        header_layout.addWidget(title)
        main_layout.addWidget(header)

        # Controls
        controls = QFrame()
        controls_layout = QHBoxLayout(controls)
        self.month_combo = QComboBox()
        months = ["January", "February", "March", "April", "May", "June", 
                 "July", "August", "September", "October", "November", "December"]
        self.month_combo.addItems(months)
        controls_layout.addWidget(QLabel("Select Month:"))
        controls_layout.addWidget(self.month_combo)
        
        update_button = QPushButton("Update Dashboard")
        update_button.clicked.connect(self.update_dashboard)
        controls_layout.addWidget(update_button)
        
        controls_layout.addStretch()
        main_layout.addWidget(controls)

        # Content Area
        content = QHBoxLayout()

        # Map Section
        map_frame = QFrame()
        map_layout = QVBoxLayout(map_frame)
        self.map_label = QLabel()
        self.map_pixmap = QPixmap("c:/Users/dhany/Smart-Grid/Karnataka_map.png")
        self.map_label.setPixmap(self.map_pixmap.scaled(700, 800, Qt.KeepAspectRatio))
        map_layout.addWidget(self.map_label)
        content.addWidget(map_frame)

        # Legend
        legend_frame = QFrame()
        legend_layout = QVBoxLayout(legend_frame)
        legend_layout.addWidget(QLabel("<b>Legend:</b>"))
        legend_layout.addWidget(QLabel("🔴 Total Power"))
        legend_layout.addWidget(QLabel("🟢 Required Supply"))
        legend_layout.addWidget(QLabel("🔵 Current Supply"))
        map_layout.addWidget(legend_frame)

        # Stats Section
        stats_frame = QFrame()
        stats_layout = QVBoxLayout(stats_frame)
        
        stats_header = QLabel("District Statistics")
        stats_header.setStyleSheet("font-size: 18px; font-weight: bold; color: #1A237E; padding: 10px;")
        stats_layout.addWidget(stats_header)
        
        stats_scroll = QScrollArea()
        stats_scroll.setWidgetResizable(True)
        stats_content = QWidget()
        self.stats_grid = QGridLayout(stats_content)
        stats_scroll.setWidget(stats_content)
        stats_layout.addWidget(stats_scroll)
        content.addWidget(stats_frame)

        main_layout.addLayout(content)

    def load_model(self):
        try:
            model_path = 'c:/Users/dhany/Smart-Grid/best_model.pth'
            if os.path.exists(model_path):
                self.model = torch.load(model_path)
                self.model.eval()
            else:
                raise FileNotFoundError(f"Model file not found at {model_path}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to load model: {str(e)}")
            sys.exit(1)

    def get_weather_data(self, district):
        try:
            url = f"http://api.openweathermap.org/data/2.5/weather"
            params = {
                'q': f"{district},Karnataka,IN",
                'appid': self.API_KEY,
                'units': 'metric'
            }
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return {
                    'temperature': data['main']['temp'],
                    'humidity': data['main']['humidity'],
                    'weather': data['weather'][0]['main'],
                    'wind_speed': data['wind']['speed']
                }
            return None
        except Exception as e:
            print(f"Weather API error for {district}: {e}")
            return None

    def predict_power(self, weather_data, month, district):
        try:
            input_features = np.zeros((1, 19))
            
            # Basic features
            input_features[0, 0] = self.le_city.transform([district])[0]
            input_features[0, 1] = self.le_state.transform(['Karnataka'])[0]
            input_features[0, 2] = month
            input_features[0, 3] = self.le_weather.transform([weather_data['weather']])[0]
            input_features[0, 4] = weather_data['temperature']
            input_features[0, 5] = weather_data['humidity']
            input_features[0, 6] = 30 if 'Rain' in weather_data['weather'] else 10
            input_features[0, 7] = weather_data['wind_speed'] * 3.6

            # Appliance power consumption
            temp = weather_data['temperature']
            input_features[0, 8] = 0.5 if temp > 25 else 0.2  # Fan
            input_features[0, 9] = 0.8 if month in [6,7,8] else 1.2  # Light
            input_features[0, 10] = 0.2  # Mixer
            input_features[0, 11] = 0.5  # Washing Machine
            input_features[0, 12] = 0.1  # Phone
            input_features[0, 13] = 0.3  # UPS
            input_features[0, 14] = 0.4  # Grinder
            input_features[0, 15] = 2.0 if temp > 30 else 0.5  # AC
            input_features[0, 16] = 1.5 if temp < 20 else 0.0  # Heater
            input_features[0, 17] = 1.5  # Fridge
            input_features[0, 18] = 0.3  # TV

            # Scale and predict
            input_scaled = self.scaler_x.transform(input_features)
            input_tensor = torch.FloatTensor(input_scaled)
            
            with torch.no_grad():
                output = self.model(input_tensor)
            
            predictions = self.scaler_y.inverse_transform(output.numpy())
            
            return {
                'total_power': float(predictions[0, 0]),
                'required_supply': float(predictions[0, 1]),
                'current_supply': float(predictions[0, 2])
            }
        except Exception as e:
            print(f"Prediction error for {district}: {e}")
            return None

    def update_dashboard(self):
        for i in reversed(range(self.stats_grid.count())): 
            self.stats_grid.itemAt(i).widget().setParent(None)

        selected_month = self.month_combo.currentIndex() + 1
        result_pixmap = QPixmap(self.map_pixmap)
        painter = QPainter(result_pixmap)
        painter.setRenderHint(QPainter.Antialiasing)

        for idx, district in enumerate(self.districts):
            weather_data = self.get_weather_data(district)
            if weather_data:
                predictions = self.predict_power(weather_data, selected_month, district)
                if predictions:
                    self.add_district_stats(idx, district, predictions, weather_data)
                    self.draw_district_indicators(painter, district, predictions)

        painter.end()
        self.map_label.setPixmap(result_pixmap.scaled(700, 800, Qt.KeepAspectRatio))

    def add_district_stats(self, idx, district, predictions, weather_data):
        district_frame = QFrame()
        district_frame.setStyleSheet("""
            QFrame {
                background-color: #E3F2FD;
                padding: 15px;
                margin: 8px;
            }
            QLabel {
                font-size: 12px;
                color: #1A237E;
            }
        """)
        layout = QVBoxLayout(district_frame)
        
        name_label = QLabel(f"<b>{district}</b>")
        name_label.setStyleSheet("font-size: 14px; color: #1565C0;")
        layout.addWidget(name_label)
        
        stats_text = f"""
        <b>Weather:</b> {weather_data['weather']}
        <b>Temperature:</b> {weather_data['temperature']:.1f}°C
        <b>Humidity:</b> {weather_data['humidity']}%
        <b>Total Power:</b> {predictions['total_power']:.2f} kWh
        <b>Required Supply:</b> {predictions['required_supply']:.2f} kW
        <b>Current Supply:</b> {predictions['current_supply']:.2f} kW
        """
        stats_label = QLabel(stats_text)
        layout.addWidget(stats_label)
        
        self.stats_grid.addWidget(district_frame, idx // 2, idx % 2)

    def draw_district_indicators(self, painter, district, predictions):
        coordinates = {
            "Bagalkot": (350, 150),
            "Ballari": (400, 250),
            "Belagavi": (250, 120),
            "Bengaluru Rural": (450, 400),
            "Bengaluru Urban": (470, 420),
            "Bidar": (500, 100),
            "Chamarajanagar": (350, 500),
            "Chikkaballapur": (450, 350),
            "Chikkamagaluru": (300, 350),
            "Chitradurga": (350, 300),
            "Dakshina Kannada": (200, 400),
            "Davanagere": (300, 300),
            "Dharwad": (280, 150),
            "Gadag": (300, 180),
            "Hassan": (280, 400),
            "Haveri": (270, 200),
            "Kalaburagi": (480, 150),
            "Kodagu": (250, 450),
            "Kolar": (500, 400),
            "Koppal": (380, 200),
            "Mandya": (350, 450),
            "Mysuru": (320, 470),
            "Raichur": (450, 200),
            "Ramanagara": (400, 420),
            "Shivamogga": (250, 300),
            "Tumakuru": (380, 350),
            "Udupi": (200, 350),
            "Uttara Kannada": (220, 200),
            "Vijayapura": (320, 100),
            "Yadgir": (470, 180),
            "Vijayanagara": (380, 230)
        }
        
        if district in coordinates:
            x, y = coordinates[district]
            
                        indicators = [
                ('total_power', QColor(255, 50, 50, 200)),     # Semi-transparent red
                ('required_supply', QColor(50, 255, 50, 200)), # Semi-transparent green
                ('current_supply', QColor(50, 50, 255, 200))   # Semi-transparent blue
            ]
            
            # Draw indicators with glow effect
            for i, (key, color) in enumerate(indicators):
                # Create glow effect
                glow_pen = QPen(color, 3)
                painter.setPen(glow_pen)
                painter.setBrush(QBrush(color))
                
                # Calculate position with spacing
                indicator_x = x + (i * 15)
                indicator_y = y
                
                # Draw main indicator
                painter.drawEllipse(indicator_x, indicator_y, 10, 10)
                
                # Add value label if needed
                value = predictions[key]
                if value > 100:  # Add warning indicator
                    painter.setPen(QPen(Qt.red, 2))
                    painter.drawEllipse(indicator_x-2, indicator_y-2, 14, 14)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    
    # Set application style
    app.setStyle('Fusion')
    
    # Create and show the main window
    window = KarnatakaPowerDashboard()
    window.show()
    
    sys.exit(app.exec_()) 

IndentationError: unexpected indent (3818841410.py, line 335)

In [ ]:


# Define the model architecture (same as training)
model = PowerConsumptionModel(input_size, output_size)  # Replace with your actual model class
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Set to evaluation mode


C:\Users\dhany\AppData\Local\Temp\ipykernel_3076\4041461524.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


PowerConsumptionModel(
  (network): Sequential(
    (0): Linear(in_features=19, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.02)
    (8): Linear(in_features=64, out_features=4, bias=True)
    (9): Softplus(beta=1.0, threshold=20.0)
  )
)